In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("longshort_strategy part 2.ipynb")

In [2]:
import datetime as dt
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings('ignore')

In [3]:
def isclose(value, original, tolerance= 0.05):
    return value <= original * (1+tolerance) and value >= original * (1-tolerance)

In [4]:
# in this homework we will
# introduce pandas through data cleaning and processing
# implement a basic trading strategy
# model the costs of a trading strategy
# calculate expected returns
# use expected returns to create a trading strategy
# try to add predictive features/signals

# returns

In [ ]:
# in order to know how any strategy would perform, we need to be able to calculate the return on investment
# in finance, we frequently use log returns rather than simple returns
# this is because log returns are easier to work with
# log returns are treated as constantly compounding return, so we can add them over time, rather than multiplying
# we can calculate the simple returns as (x1-x0)/x0, the percentage change
# log returns are calculated as log(x1/x0)
# see if you can derive the transformations from log returns to simple returns, and simple returns to log returns
hdf = pd.read_parquet('stock_data.parquet')

In [ ]:
# use np.exp
def logtosimple(logreturn):
    return ...

In [ ]:
grader.check("q3a")

In [ ]:
# use np.log
def simpletolog(simplereturn):
    return ...

In [ ]:
grader.check("q3b")

In [ ]:
# let's look at an example asset
# create a dataframe that is just the adjusted close and volume for AAL
aal = hdf.loc[:,hdf.columns.get_level_values(1) == ...]

In [ ]:
grader.check("q3c")

In [ ]:
# let's remove the top level of the columns in order to make things easier - we already know the symbol is AAL
aal.columns = aal.columns.get_level_values(0)

In [ ]:
grader.check("q3d")

In [ ]:
# we can create a new column using
# df['newcolname'] = series
# let's create a new column for the returns in AAL
# we can get the returns using .pct_change() on the adjusted close column
aal['returns'] = ...

In [ ]:
grader.check("q3e")

In [ ]:
# we can get the log returns by adjusting the percent change function

aal['logreturns'] = ...

In [ ]:
grader.check("q3f")

In [ ]:
# .cumsum() on a series will add everything in the column up until that point
# this will give us the log return from the start, to the current index
aal['cum_logreturns'] = ...

In [ ]:
grader.check("q3g")

In [ ]:
# pandas also comes equipped with some built in plotting
# we can plot the adjusted close, as well as the logreturns cumsum, and see if they look the same (they should)
aal['Adj Close'].plot()

In [ ]:
aal['cum_logreturns'].plot()

# basic trend strategy: long/short x%-ile

In [ ]:
# now we are going to try to create a trading strategy
# we'll assume that we can short, and assume that the cost to borrow is 0 (not trivial assumption)
# our trading strategy is going to try to buy stocks that will go up, and sell stocks that will go down
# one way to do this is to try to rank the stocks based on some metric
# and go long/go short the top/bottom percent by that metric

In [ ]:
hdf

In [ ]:
# to more easily group by asset, we'll make assets into its own column using df.stack()
hdf_old = hdf.copy()
hdf = (
    hdf.stack()
    .reset_index()
    .rename(columns={'level_1': 'Symbol'})
)
hdf

In [ ]:
# now we calculate returns and log returns for the entire dataframe
# we can **group by the ticker**, select the adjusted close column, and then use .pct_change() to get the percentage change from one row to the next
hdf['returns'] = ...

In [ ]:
grader.check("q4a")

In [ ]:
# we'll also create a log returns column
# we do this by creating the logreturns column using our simple to log function
hdf['logreturns'] = ...

In [ ]:
grader.check("q4b")

In [ ]:
# we want to predict the next step of returns, so we need to create a column for that 
# calculate the forward logreturn by **grouping by the ticker**, selecting the logreturns column,
# and then shifting the series up a row with .shift(-1)

hdf['fwd_logreturn'] = ...

In [ ]:
grader.check("q4c")

In [ ]:
# let's also drop all of the rows where logreturns and forward logreturns have NaN values, as these will mess up future operations
# we can do this with the subset parameter in df.dropna(subset=[])
hdf = hdf.dropna(subset=[...])

In [ ]:
grader.check("q4d")

# momentum

In [ ]:
# now we can rank the returns by each asset
# what metric should we use? 

# you might notice that stocks that went up tend to continue to go up, 
# and stocks that go down tend to continue to go down
# this economically makes sense because people who see that a stock went up by a lot might buy it to join the trend, and vice versa

# so we can figure out which assets went up the most, and buy those
# and we can short the assets that went down the most

In [ ]:
# we can calculate this using the .rank() method
# it will output the rank value, if the dataframe was sorted 
# we'll use 'ascending=False', in order to make the
# assign the logreturn_rank column to a rank of the logreturns column, grouped by date
# be sure to use .rank(method='dense')

hdf['logreturn_rank'] = ...

In [ ]:
grader.check("q5a")

In [ ]:
# Calculate the mean log returns for each date
date_mean_logreturns = hdf.groupby('Date')['logreturns'].transform('mean')

# Calculate the demeaned log returns
hdf['demeaned_logreturn'] = hdf['logreturns'] - date_mean_logreturns

# Calculate the mean forward log returns for each date
date_mean_fwd_logreturns = hdf.groupby('Date')['fwd_logreturn'].transform('mean')

# Calculate the demeaned forward log returns
hdf['demeaned_fwd_logreturn'] = hdf['fwd_logreturn'] - date_mean_fwd_logreturns

In [ ]:
# we can also look at the demeaned returns, or the returns of the asset we chose relative to all assets
# this will help us assess if we were able to choose the assets that would end up under/overperforming

date_mean_logreturns = hdf.groupby('Date')['logreturns'].transform('mean')

# Calculate the demeaned log returns
hdf['demeaned_logreturn'] = hdf['logreturns'] - date_mean_logreturns

# Calculate the mean forward log returns for each date
date_mean_fwd_logreturns = ...

# Calculate the demeaned forward log returns
hdf['demeaned_fwd_logreturn'] = ...

In [ ]:
grader.check("q5b")

In [ ]:
# for ease of analyzing the effect, we will bucket our feature using a decile (each bin is 10%-ile)
# this will help us figure out how strong the effect is
# we do this using pd.qcut(x, q=quantile, labels=False, duplicates='drop'), which cuts a series into quantile buckets. pass q=10 to get decile buckets
# drop labels and duplicates
hdf['logreturn_decile'] = hdf.groupby('Date')['logreturns'].transform(
    lambda x: pd.qcut(x, q=10, labels=False, duplicates='drop'))

In [ ]:
# we'll plot this using a barplot
hdf.groupby('logreturn_decile').mean(numeric_only = True)['demeaned_fwd_logreturn'].plot(kind='bar')

In [ ]:
# it definitely looks like the bottom and top 20%-ile have outsized returns!

# strategy 'backtesting'

In [ ]:
# while backtesting can be a useful tool, its often a bit overkill when making an initial test to see if a feature is useful
# here we will do a very basic backtest: assuming no trading costs
# costs to trade are a huge drag on our edge, but not worth thinking about at least for now
# the idea is that if it doesnt work when there are no costs, it probably won't work when there are costs

In [ ]:
# let's define our strategy by setting an asset to be long or short if its in the top or bottom 20% log return decile
# we'll also equal weight all our positions
# the decile values for the bottom 20% would be 0 and 1
# the decile values for the top 20% would be 8 and 9

# we can use np.where() in order to conditionally assign a column
hdf['long/short'] = ...

In [ ]:
grader.check("q6a")

In [ ]:
# we size our positions by taking our absolute long/short position
hdf['absposition'] = ...

In [ ]:
grader.check("q6b")

In [ ]:
# getting the total number of our positions by summing our absolute position for each day 
hdf['numpositions'] = ...

In [ ]:
grader.check("q6c")

In [ ]:
# and we get our final weight for each asset by scaling our long/short indicator variable by the number of positions we have
# each position should be such that we add up to one, so we'd divide the indicator by total positions
# if we have 0 positions, the weight should be 0
# this will mean that we should have equal size long and short, adding up to a total of 1 (no leverage)
# we'll use df.apply(lambda row: (expression using row) if (condition on some row) else value, axis=1)

hdf['weight'] = ...

In [ ]:
grader.check("q6d")

In [ ]:
# finally we define the strategy's return as the weighted logreturns based on our position
# so we multiply weight by forward logreturn
hdf['strategy_logreturn'] = ..

In [ ]:
grader.check("q6e")

In [ ]:
# we can compare this with an equal weighted strategy, where we long each asset the same amount, again summing up to 1
hdf['eqweight'] = ...
hdf['equal_logreturn'] = ...

In [ ]:
grader.check("q6f")

# evaluating performance

In [ ]:
hdf.groupby('Date').sum()['strategy_logreturn'].cumsum().plot(legend=True)
hdf.groupby('Date').sum()['equal_logreturn'].cumsum().plot(legend=True)

In [ ]:
# we can get the final return by getting the last value of the column (values[-1])
# and we can translate that to simple returns for readability
final_strategyreturn = logtosimple(hdf.groupby('Date').sum()['strategy_logreturn'].cumsum().values[-1])
final_equalreturn = ...

print(f'final_strategyreturn: {final_strategyreturn}')
print(f'...')

In [ ]:
grader.check("q7a")

In [ ]:
# the equal strategy way outperformed our strategy! since our strategy was rather simple, this isn't too unexpected
# if you notice, the equal strategy was rather volatility

# one way to account for this is using a sharpe ratio
# a sharpe ratio is essentially a t-test for the statistical significance of a strategy's risk adjusted return
# its the mean return of the strategy minus the risk free rate, divided by the standard deviation of return of the strategy

# generally, any sharpe ratio over 1 is good, 2 is very good, 3+ is very very good
# as you see a sharpe ratio > 3, the more likely that the strategy is somehow limited, or you've calculated something wrong
# there are many simplifications in this code, so don't worry about the sharpe ratios, they are not realistic
# as an example: limiting our universe to companies that are currently in the SP500 introduces a lookahead bias
# we are using information (stocks still listed in the sp500 as of today) that we wouldnt have had at the time (back in 2023)
# try to think of more ways this sample or testing strategy might be limited!

In [ ]:
# let's calculate the sharpe ratio with a function, we'll leave out the risk free rate part of it for now
# we also need to normalize the sharpe ratio with respect to a year, by multiplying by the square root of periods our strategy trades in a year
# note that there are 252 trading days

def sharpe_ratio(mean_ret, std_ret):
    return ... * np.sqrt(...)
    
strat_sharpe = sharpe_ratio(hdf.groupby('Date').sum()['strategy_logreturn'].mean(), 
             hdf.groupby('Date').sum()['strategy_logreturn'].std())
equal_sharpe = sharpe_ratio(..., 
             ...)
print(strat_sharpe, equal_sharpe)

In [ ]:
grader.check("q7c")

In [ ]:
# interesting: it looks like the equal strategy had a very high sharpe
# this is because 2023 oct to 2024 march was a very bullish time (you can look up the S&P 500 returns back then to check)
# usually, we would want to backtest over a longer time period, such as over a year, to reduce variance from such market idiosyncrasies

# accounting for fees

In [ ]:
# this is without fees so it is clearly way too good
# let's add a fee for each trade, and expected slippage per trade
# the fee is what we would pay to the broker, and the expected slippage is likely a function of our position size
# we'll combine these into one value, and just observe how our strategy decays as a function of cost

# let's define a percentage fee per trade (e.g., 0.02%)
fee = 0.0002

In [ ]:
# in order to see how large our trade would be, we have to find the difference between our previous and current position size
# we should sort values by symbol, then by the date
hdf = hdf.sort_values(by=[...]) # order matters!

# we'll groupby symbol, and then get the previous weight by using .shift(1) to shift the weights down
hdf['prevweight'] = hdf.groupby('Symbol')['weight'] ...

# next, we'll get the strategy's weight change by taking the difference between weight and prevweight
hdf['strategy_weightchange'] = ...

# finally, to calculate fees, we'll need to multiply the fee by our absolute change in position
hdf['strategy_fees'] = abs(hdf['strategy_weightchange']) * fee

In [ ]:
grader.check("q8a")

In [ ]:
# now we'll calculate the strategy log return after fees by subtracting the groupby fees from the groupby logreturn
hdf['strategy_postfees'] = ...
strategy_postfees_seriestoplot = ...
strategy_postfees_seriestoplot.plot()

In [ ]:
grader.check("q8b")

In [ ]:
# and we'll test the sharpe and logreturn as above
fees_sharpe = sharpe_ratio(..., 
                           ...)

In [ ]:
grader.check("q8c")

In [ ]:
# note that the equal weight buy and hold does not change with fees, as it never changes position

# expected returns

In [ ]:
# now that we have a very basic long/short strategy, we should try to improve upon it

# our strategy roughly was equally long and short the market - regardless of how strongly something moved
# even if all of the longs were very high return, the strategy didn't care
# it also didn't care if it was 10th or 20th decile: we gave the same weight regardless
# we might expect that there's a way to improve upon this

# one way of doing this is trying to calculate an 'expected return' for each asset
# this allows us to weight our positions based on how good we think they are

In [ ]:
# how might we make an expected returns model? 
# we'll likely want to fit to some historical data, and see how that strategy performs on data after that

# to avoid our model just learning the optimal answer for our entire dataset
# we'll train the model on the first 80% of our data
# and see how it performs on the remaining 20%

In [ ]:
# split the data into training and testing using .iloc
train_percent = ... # use 80% as a decimal
# make sure to split according to time series!

hdf = hdf.sort_values(...) # order matters! we need to split by time first, then asset

# we can only use integer indices, so make sure to cast the value to an integer
splitrow = ...
training = hdf.iloc[:splitrow]
testing = hdf.iloc[splitrow:]



In [ ]:
grader.check("q9a")

In [ ]:
# a very common basic model is a linear regression: fitting a line to points of data
# given some x variable, we try to solve for the optimal y = mx + b, 
# we do this by minimizing the squared sum of differences of our line to each data point
# luckily there are libraries that do this for us

In [ ]:
# to run a linear regression on our training data, we need a data matrix X of features
# and a target y to fit to 
# in our case, our target is forward log return
# and our data matrix X is the current log returns
# let's run the regression using statsmodels

In [ ]:
feature = [...]
target = [...]

# we add a constant to data matrix Xin order to get an intercept term, otherwise we would be fitting y = mx
X = training[...]
X = sm.add_constant(X)

y = training[...]

model = sm.OLS(y, X).fit()
print(model.params)

In [ ]:
grader.check("q9b")

In [ ]:
# now we can plot our linear regression

m = model.params[...]
b = model.params[...]

# Create a scatter plot
plt.scatter(training[feature], y, color='blue', label='Data')

# Plot the linear regression line
plt.plot(training[feature], m * training[feature] + b, color='red', label='linear regression')

In [ ]:
grader.check("q9c")

In [ ]:
# now we can see how well our model does on our testing data
X_test = sm.add_constant(testing[feature])
y_pred = model.predict(...).values

# we compare the predictions to the actual values
y_test = testing[...].values

# we use mean squared error: the mean difference between y_test and y_pred, squared
mse = ...

In [ ]:
grader.check("q9d")

# updating backtest 

In [ ]:
# we also can look at our model's sharpe ratio on the testing data

In [ ]:

# we can get the expected log returns by applying our model, with a constant, to the feature column

testing['ex_logreturns'] = model.predict(sm.add_constant(...))

# we'll size our weights according to the cross sectional predictions
# use transform again to do this
# we want to have each weight be (ex_logreturn-mean ex_logreturn for date)/(sum of absolute ex_logreturns for date)

testing['ex_weight'] = testing.groupby('Date')['ex_logreturns'].transform(lambda x: ...)

# and we multiply the forward log returns again
testing['exstrategy_logreturns'] = ...

# and we plot once more
testing.groupby('Date').sum()['exstrategy_logreturns'].cumsum().plot()

In [ ]:
grader.check("q10a")

In [ ]:
# wow that looks very promising! let's calculate the sharpe ratio with fees
# sort again
testing = testing.sort_values(...)
# get previous ex_weight as before
testing['prevex_weight'] = ...
# get change in weight as before
testing['exstrategy_weightchange'] = ...

# get absolute change in position
testing['exstrategy_fees'] = ...
# and find the return post fees
testing['exstrategy_postfees'] = ...


In [ ]:
grader.check("q10b")

# feature engineering

In [ ]:
# let's try to add more features to see if we can make this model any better

In [ ]:

# first, let's create a function that makes the bar plot from before, so we can easily view any feature
def summarize_feature(hdf, colname):
    if hdf[colname].dtype == 'float':
        # use qcut here, by deciles as before
        bins = ...
        hdf.groupby(bins).mean()['demeaned_fwd_logreturn'].plot(kind='bar')
    else:
        hdf.groupby(colname).mean()['demeaned_fwd_logreturn'].plot(kind='bar')

In [ ]:
grader.check("q11a")

In [ ]:
# let's reincorporate sector data into this dataframe
# we can do this using the merge function in pandas
columns
hdf = hdf.merge(df[[columns]], on='Symbol', how='left')

In [ ]:
grader.check("q11b")

In [ ]:
# another possible feature idea could be notional volume, or the dollar amount of shares traded
# we can get notional volume by multiplying the close price with the shares traded
hdf['ntlvolume'] = ...

In [ ]:
grader.check("q11c")

In [ ]:
# feel free to create more features here, and use the below functions to test them!

# testing!

In [ ]:
def load_model(hdf, features, train_test_split=0.8, debug=0):
    # split the data into training and testing using .iloc
    train_percent = 0.8
    
    # make sure to split according to time series!
    hdf = hdf.sort_values(['Date', 'Symbol'])
    training = hdf.iloc[:int(len(hdf) * train_percent)]
    testing = hdf.iloc[int(len(hdf) * train_percent):]
    
    target = ['fwd_logreturn']
    
    # Identify categorical features based on data type
    categorical_features = training[features].select_dtypes(include=['object', 'category']).columns.tolist()
    
    # one hot encoding for categorical features
    # we essentially create a bunch of extra columns, and assign them as ones and zeros
    training = pd.get_dummies(training, columns=categorical_features)
    testing = pd.get_dummies(testing, columns=categorical_features)
    
    # Update the features list to include dummy variables
    features = [col for col in training.columns if col in features or col.startswith(tuple(categorical_features))]
    if debug > 0:
        print(f'features: {features}')
    X_train = training[features]
    X_train = sm.add_constant(X_train)
    y_train = training[target]
    
    model = sm.OLS(y_train, X_train).fit()
    if debug > 0:
        print(f'r_squared: {model.rsquared}')
    
    # For testing data
    X_test = testing[features]  # Include the same features used for training
    X_test = sm.add_constant(X_test)
    
    return testing, model, X_test, debug  # Return testing data along with the model and testing features

def basic_backtest(testing, model, X_test, debug, fee=0.0002):
    testing['ex_logreturns'] = model.predict(X_test)  # Use X_test for prediction
    testing['ex_weight'] = testing.groupby('Date')['ex_logreturns'].transform(lambda x: (x - x.mean()) / x.abs().sum())
    testing['exstrategy_logreturns'] = testing['ex_weight'] * testing['fwd_logreturn']
    testing = testing.sort_values(by=['Symbol', 'Date'])
    testing['prevex_weight'] = testing.groupby('Symbol')['ex_weight'].shift(1)
    testing['exstrategy_weightchange'] = testing['ex_weight'] - testing['prevex_weight']
    testing['exstrategy_fees'] = abs(testing['exstrategy_weightchange']) * fee
    testing['exstrategy_postfees'] = testing['strategy_logreturn'] - testing['exstrategy_fees']
    if debug > 0:
        testing.groupby('Date').sum()['exstrategy_fees']
    testing.groupby('Date').sum()['exstrategy_postfees'].cumsum().plot()
    print(f"sharpe_ratio: {sharpe_ratio(testing.groupby('Date').sum()['exstrategy_postfees'].mean(), testing.groupby('Date').sum()['exstrategy_postfees'].std())}")
    if debug > 1:
        plt.figure(figsize=(12, 8))
        for symbol in testing['Symbol'].unique():
#             print(symbol)
            asset_weights = testing[testing['Symbol'] == symbol].set_index('Date')['ex_weight']
            asset_weights.plot(label=symbol)
        plt.title('Rolling Weights of Assets')
        plt.xlabel('Date')
        plt.ylabel('Weight')
        plt.legend()
        plt.show()

In [ ]:
# use the debug levels to get more or less information
# pass your features into the list here
# be very careful when changing the above functions!
features = ['logreturns']
basic_backtest(*(load_model(hdf, features, debug=1)), fee=0.0002)

In [ ]:
# there's much more to cover in the realm of researching and testing systematic trading strategies
# importantly, we haven't covered much about risk modeling, and portfolio optimization 
# check out https://www.alacra.com/alacra/help/barra_handbook_US.pdf 
# for a good introduction to risk modeling, if you're curious!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Please also check gradescope for any written assignments for this week.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)